<a href="https://colab.research.google.com/github/Ishank2301/Comprehensive-ML-Handbook/blob/main/Cats_and_Dog_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries:

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

tf.config.run_functions_eagerly(True)

# Generate Data:

### 1. Install the Kaggle library

In [2]:
pip install kaggle

### 2. Configure your Kaggle API key

Download your `kaggle.json` file from your Kaggle account (Profile -> Account -> Create New API Token).

Then, upload this file to your Colab environment in the directory `~/.kaggle/` and set the correct permissions.

In [3]:
import os
from google.colab import files


# Upload kaggle.json
files.upload()


print("Kaggle API key configured successfully!")

Saving kaggle.json to kaggle.json
Kaggle API key configured successfully!


In [4]:
# Create .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle
# Move the uploaded kaggle.json to the correct directory
!mv kaggle.json ~/.kaggle/

# Set permissions
!chmod 600 ~/.kaggle/kaggle.json


### 3. Import `kagglehub` and download the dataset



In [5]:
import kagglehub
!kaggle datasets download -d bhavikjikadara/dog-and-cat-classification-dataset
!unzip dog-and-cat-classification-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: PetImages/Dog/5499.jpg  
  inflating: PetImages/Dog/55.jpg    
  inflating: PetImages/Dog/550.jpg   
  inflating: PetImages/Dog/5500.jpg  
  inflating: PetImages/Dog/5501.jpg  
  inflating: PetImages/Dog/5502.jpg  
  inflating: PetImages/Dog/5503.jpg  
  inflating: PetImages/Dog/5504.jpg  
  inflating: PetImages/Dog/5505.jpg  
  inflating: PetImages/Dog/5506.jpg  
  inflating: PetImages/Dog/5507.jpg  
  inflating: PetImages/Dog/5508.jpg  
  inflating: PetImages/Dog/5509.jpg  
  inflating: PetImages/Dog/551.jpg   
  inflating: PetImages/Dog/5510.jpg  
  inflating: PetImages/Dog/5511.jpg  
  inflating: PetImages/Dog/5512.jpg  
  inflating: PetImages/Dog/5513.jpg  
  inflating: PetImages/Dog/5514.jpg  
  inflating: PetImages/Dog/5515.jpg  
  inflating: PetImages/Dog/5516.jpg  
  inflating: PetImages/Dog/5517.jpg  
  inflating: PetImages/Dog/5518.jpg  
  inflating: PetImages/Dog/5519.jpg  
  inflating: PetImages/Dog/552.jpg   

# Formatting the Data:

In [6]:
import os
import shutil

# Define base directory
base_dir = '/content/PetImages'

# Define new train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create Cat and Dog subdirectories within train and test
os.makedirs(os.path.join(train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Dog'), exist_ok=True)

print(f"Created directories: {train_dir}, {test_dir}")

Created directories: /content/PetImages/train, /content/PetImages/test


In [26]:
import random
import shutil
from PIL import Image
import os # Import os module for file size check

# Redefine check_image to be self-contained for this cell
def check_image(path):
    # First, check if the file is empty
    if not os.path.exists(path):
        # print(f"Corrupted image detected (not copied): {path}, Error: File does not exist.")
        return False
    if os.path.getsize(path) == 0:
        print(f"Corrupted image detected (not copied): {path}, Error: Zero-byte file.")
        return False
    try:
        with Image.open(path) as img:
            img.verify() # Verify file integrity
            img.load()   # Load image data to catch more errors
            # Attempt to convert to RGB to handle problematic channel counts and other issues
            img = img.convert('RGB')
            return True
    except Exception as e:
        print(f"Corrupted image detected (not copied): {path}, Error: {e}") # Activate for verbose debugging
        return False

# Define the paths to the original Cat and Dog images
original_cat_dir = os.path.join(base_dir, 'Cat')
original_dog_dir = os.path.join(base_dir, 'Dog')

# Get all VALID image file names from original directories
cat_fnames_all = [fname for fname in os.listdir(original_cat_dir) if fname.endswith('.jpg') and check_image(os.path.join(original_cat_dir, fname))]
dog_fnames_all = [fname for fname in os.listdir(original_dog_dir) if fname.endswith('.jpg') and check_image(os.path.join(original_dog_dir, fname))]

# Shuffle filenames
random.seed(42) # for reproducibility
random.shuffle(cat_fnames_all)
random.shuffle(dog_fnames_all)

# Define split ratio (e.g., 80% train, 20% test)
train_split_ratio = 0.8

# Split Cat images
train_cat_fnames = cat_fnames_all[:int(len(cat_fnames_all) * train_split_ratio)]
test_cat_fnames = cat_fnames_all[int(len(cat_fnames_all) * train_split_ratio):]

# Split Dog images
train_dog_fnames = dog_fnames_all[:int(len(dog_fnames_all) * train_split_ratio)]
test_dog_fnames = dog_fnames_all[int(len(dog_fnames_all) * train_split_ratio):]

print(f"Total valid Cat images: {len(cat_fnames_all)}, Train Cat: {len(train_cat_fnames)}, Test Cat: {len(test_cat_fnames)}")
print(f"Total valid Dog images: {len(dog_fnames_all)}, Train Dog: {len(train_dog_fnames)}, Test Dog: {len(test_dog_fnames)}")

# Function to copy files
def copy_files(fnames, src_dir, dest_dir):
    for fname in fnames:
        src_path = os.path.join(src_dir, fname)
        dest_path = os.path.join(dest_dir, fname)
        try:
            shutil.copyfile(src_path, dest_path)
        except Exception as e:
            print(f"Error copying {fname}: {e}") # This should ideally not happen after filtering

# Clear existing train and test subdirectories before populating with valid images
shutil.rmtree(os.path.join(train_dir, 'Cat'), ignore_errors=True)
shutil.rmtree(os.path.join(train_dir, 'Dog'), ignore_errors=True)
shutil.rmtree(os.path.join(test_dir, 'Cat'), ignore_errors=True)
shutil.rmtree(os.path.join(test_dir, 'Dog'), ignore_errors=True)

# Recreate empty subdirectories
os.makedirs(os.path.join(train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Dog'), exist_ok=True)

# Copy images to train directories
copy_files(train_cat_fnames, original_cat_dir, os.path.join(train_dir, 'Cat'))
copy_files(train_dog_fnames, original_dog_dir, os.path.join(train_dir, 'Dog'))

# Copy images to test directories
copy_files(test_cat_fnames, original_cat_dir, os.path.join(test_dir, 'Cat'))
copy_files(test_dog_fnames, original_dog_dir, os.path.join(test_dir, 'Dog'))

print("Valid images moved to train and test directories.")

Streaming output truncated to the last 5000 lines.
Corrupted image detected (not copied): /content/PetImages/Cat/4116.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/4380.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/5452.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/9242.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/6021.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/10081.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/5044.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Cat/9844.jpg, Error: 'NoneType' object has 

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Streaming output truncated to the last 5000 lines.
Corrupted image detected (not copied): /content/PetImages/Dog/9253.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/1861.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/3680.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/2337.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/10538.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/7176.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/9934.jpg, Error: 'NoneType' object has no attribute 'seek'
Corrupted image detected (not copied): /content/PetImages/Dog/4539.jpg, Error: 'NoneType' object has 

In [27]:
# Generators
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/PetImages/train',
    labels = 'inferred',
    label_mode= 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/content/PetImages/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 0 files belonging to 2 classes.


ValueError: No images found in directory /content/PetImages/train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

# Normalizing our data:


In [ ]:
import os
from PIL import Image

def check_image(path):
    try:
        with Image.open(path) as img:
            img.verify() # Verify file integrity
            img.load()   # Load image data to catch more errors

            # Explicitly check if the image has 2 channels
            if len(img.getbands()) == 2:
                raise ValueError(f"Image has 2 channels (mode: {img.mode}), which is not supported.")

            return True
    except Exception as e:
        print(f"Corrupted image found and removed: {path}, Error: {e}")
        return False

def clean_directory(directory):
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            print(f"Cleaning directory: {subdir_path}")
            for fname in os.listdir(subdir_path):
                fpath = os.path.join(subdir_path, fname)
                if not check_image(fpath):
                    os.remove(fpath)

# The cleaning calls are removed here as filtering now happens during the split
# clean_directory('/content/PetImages/train')
# clean_directory('/content/PetImages/test')

# Normalizing our data:
def process(image,label):
  image = tf.cast(image/255, tf.float32)
  return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

# Create our Model:

In [10]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3), padding='same', activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,878,913 (64.39 MB)

 Trainable params: 16,878,913 (64.39 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
ES = EarlyStopping(monitor="val_loss",
                   min_delta=0.00001,
                   patience=50,
                   verbose=1,
                   mode="auto",
                   baseline=None,
                   restore_best_weights=True,
                   )

In [14]:
history = model.fit(train_ds, epochs=10, validation_data = (validation_ds),verbose=1,callbacks=ES)

Epoch 1/10
110/625 ━━━━━━━━━━━━━━━━━━━━ 1:19 154ms/step - accuracy: 0.5432 - loss: 0.9707

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Number of channels inherent in the image must be 1, 3 or 4, was 2
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

In [17]:
history = model.fit(train_ds, epochs=10, validation_data = (validation_ds),verbose=1,callbacks=ES)

Epoch 1/10
 84/625 ━━━━━━━━━━━━━━━━━━━━ 1:30 168ms/step - accuracy: 0.5107 - loss: 0.6929

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Number of channels inherent in the image must be 1, 3 or 4, was 2
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

In [ ]:
# The 'plot_decision_regions' function is typically used for classification tasks with 2D data.
# Since we are performing a regression task with multiple features, this plot is not suitable.
# I will comment out this code and suggest an alternative visualization for regression results if needed.
# plot_decision_regions(X,y.astype('int'),clf=Model,legend=2)
# plt.xlim(-2,3)
# plt.xlim(-1.5,2)
# plt.show()

** I think using all these methods like Dropout, EarlyStopping and Regularization did make our model more accurate and saved time and computation and our model is more or less perfect fit

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])